In [1]:
import pyensembl 

In [2]:
human = pyensembl.genome_for_reference_name("GRCh38")
mouse = pyensembl.genome_for_reference_name("GRCm38")

In [3]:
human_transcripts = human.transcript_sequences.fasta_dictionary
mouse_transcripts = mouse.transcript_sequences.fasta_dictionary

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/iskander/Library/Caches/pyensembl/GRCh38/ensembl109/Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/iskander/Library/Caches/pyensembl/GRCh38/ensembl109/Homo_sapiens.GRCh38.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/iskander/Library/Caches/pyensembl/GRCm38/ensembl102/Mus_musculus.GRCm38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/iskander/Library/Caches/pyensembl/GRCm38/ensembl102/Mus_musculus.GRCm38.ncrna.fa.gz.pickle


In [4]:
test_set = {'AACCCCTTTGAAACTTGATCC',
 'AAGGAGATCCCACCAGAAGCT',
 'AAGGAGTATGTTGACATCATG',
 'ACATCATGGAATGGCTGGTGG',
 'ACTCACTTGGCCACTGGGGAG',
 'AGCAGCCAGTGTACATTCCGA',
 'AGCCAGTGTACATTCCGAAGA',
 'AGGAAGGGATGTATCCAGATC',
 'AGGAGGAAGGGATGTATCCAG',
 'AGGAGTATGTTGACATCATGG',
 'AGTATGTTGACATCATGGAAT',
 'AGTGCTTCGTTCCCTGGCCCG',
 'AGTGGGAGCACACCAGCAACT',
 'AGTGTACATTCCGAAGAAGGC',
 'ATTGGCTGTGCAGGAGTGGGA',
 'CAACCCCTTTGAAACTTGATC',
 'CAAGGAGATCCCACCAGAAGC',
 'CAGCAGCCAGTGTACATTCCG',
 'CAGCCAGTGTACATTCCGAAG',
 'CAGTGCTTCGTTCCCTGGCCC',
 'CAGTGTACATTCCGAAGAAGG',
 'CATCAATGAGCTGTGTTCTCA',
 'CCAGCAGCCAGTGTACATTCC',
 'CCAGTGCTTCGTTCCCTGGCC',
 'CCAGTGTACATTCCGAAGAAG',
 'CCCAGTGCTTCGTTCCCTGGC',
 'CCCCCCGCCGGCGTCAGCGTA',
 'CCCCCGCCGGCGTCAGCGTAA',
 'CCCCGCCGGCGTCAGCGTAAA',
 'CCCGCCGGCGTCAGCGTAAAG',
 'CCGCCGGCGTCAGCGTAAAGC',
 'CCGGCGTCAGCGTAAAGCCCA',
 'CCTGAGGCACCCAAGGAGATC',
 'CGCCGGCGTCAGCGTAAAGCC',
 'CGGCGTCAGCGTAAAGCCCAG',
 'CGTCAGCGTAAAGCCCAGAGA',
 'GAAGGAGTATGTTGACATCAT',
 'GACCGGATGATCTTTTATGAG',
 'GACTCACTTGGCCACTGGGGA',
 'GAGGAAGGGATGTATCCAGAT',
 'GAGTATGTTGACATCATGGAA',
 'GAGTGGGAGCACACCAGCAAC',
 'GCAACCCCTTTGAAACTTGAT',
 'GCAGCCAGTGTACATTCCGAA',
 'GCCAGTGTACATTCCGAAGAA',
 'GCCCACTATGACCCTGGAGGA',
 'GCCCCCCGCCGGCGTCAGCGT',
 'GCCGGCGTCAGCGTAAAGCCC',
 'GCGTCAGCGTAAAGCCCAGAG',
 'GCTTCGTTCCCTGGCCCGGCT',
 'GGAAGGGATGTATCCAGATCC',
 'GGAGGAAGGGATGTATCCAGA',
 'GGAGTATGTTGACATCATGGA',
 'GGCCCCCCGCCGGCGTCAGCG',
 'GGCGTCAGCGTAAAGCCCAGA',
 'GTATGTTGACATCATGGAATG',
 'GTCAGCGTAAAGCCCAGAGAC',
 'GTGAAGGAGTATGTTGACATC',
 'GTGCTTCGTTCCCTGGCCCGG',
 'GTGTACATTCCGAAGAAGGCA',
 'TCCTGAGGCACCCAAGGAGAT',
 'TGAAGGAGTATGTTGACATCA',
 'TGCAACCCCTTTGAAACTTGA',
 'TGCTTCGTTCCCTGGCCCGGC',
 'TTGGCTGTGCAGGAGTGGGAG'}

test_string = "".join(test_set)


In [5]:
import tqdm
from collections import defaultdict
import numpy as np

from sys import intern 

def seq_to_kmers(seq, k):
    return {
        seq[i:i+k] for i in range(len(seq) - k + 1) 
    }

In [7]:
from sys import intern

def record_kmer_sources(transcripts, min_k=12, max_k=14):
    ks = list(range(min_k, max_k + 1))
    kmers = {k: defaultdict(set) for k in ks}
    max_kmers = defaultdict(list)
    print("k=%d" % max_k)
    for t_id, seq in tqdm.tqdm(transcripts.items()):
        for kmer in seq_to_kmers(np.string_(seq), max_k):
            max_kmers[kmer].append(t_id)
    # kmer[max_k] = {kmer: set(t_ids) for (kmer, t_ids) in max_kmers.items()}
    kmers[max_k] = max_kmers
    for k in range(max_k - 1, min_k - 1, -1):
        print("...k=%d" % k)
        kmers_one_higher = kmers[k + 1]
        curr_kmers = kmers[k]
        for (kmer, transcript_ids) in tqdm.tqdm(kmers_one_higher.items()):
            curr_kmers[kmer[1:]].update(transcript_ids)
            curr_kmers[kmer[:-1]].update(transcript_ids)
    return kmers 

min_k = 14
max_k = 14
print("== Human ==")
human_kmers = record_kmer_sources(human_transcripts, min_k, max_k) 

print("== Mouse ==")
mouse_kmers = record_kmer_sources(mouse_transcripts, min_k, max_k) 

== Human ==
k=14


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 273426/273426 [35:34<00:00, 128.10it/s]


== Mouse ==
k=14


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 141450/141450 [5:29:34<00:00,  7.15it/s]


In [ ]:
human_nutm1 = human.genes_by_name("NUTM1")[0]
mouse_nutm1 = mouse.genes_by_name("Nutm1")[0]
human_nutm1_transcripts = human_nutm1.transcripts
human_nutm1_transcript_ids = {t.id for t in human_nutm1_transcripts}
mouse_nutm1_transcripts = mouse_nutm1.transcripts
mouse_nutm1_transcript_ids = {t.id for t in mouse_nutm1_transcripts}


In [ ]:
human_nutm1_seqs = {t.id: t.sequence for t in human_nutm1_transcripts}

In [ ]:
{t_id: len(seq) for (t_id, seq) in human_nutm1_seqs.items()}

In [ ]:
from collections import Counter, defaultdict
aso_size = 21
nt_from_last_exon = 1000
nutm1_transcript_id_to_kmers = {}
nutm1_transcript_id_to_kmer_counts = {}
kmer_to_sources = defaultdict(set)
for t in human_nutm1_transcripts:
    print(t.name, t.id, len(t.exons), [len(e) for e in t.exons])
    exons = t.exons
    
    sequence = t.sequence
    start_offset = len(exons[0]) + len(exons[1])
    middle_exons_length = sum([len(e) for e in exons[2:-1]])
                         
    end_offset = start_offset + middle_exons_length + nt_from_last_exon
    print("... %d:%d" % (start_offset, end_offset))
    subseq = sequence[start_offset:end_offset]
    kmers = seq_to_kmers(subseq, aso_size)
    nutm1_transcript_id_to_kmers[t.id] = kmers
    c = Counter()
    for kmer in kmers:
        c[kmer] += 1
    nutm1_transcript_id_to_kmer_counts[t.id] = c

intersection_kmers = set.intersection(*nutm1_transcript_id_to_kmers.values())
ok_ids = {t.id for t in human_nutm1_transcripts}

kmer_to_off_target_count = Counter()
for candidate in tqdm.tqdm(intersection_kmers):
    n = len(candidate)
    for i in range(n - max_k + 1):
        sub_kmer = candidate[i:i+max_k]
        off_target_transcript_ids = set(human_kmers[max_k][sub_kmer])
        off_target_transcript_ids_without_nutm1 = off_target_transcript_ids.difference(ok_ids)
        kmer_to_off_target_count[candidate] += len(off_target_transcript_ids_without_nutm1)

In [ ]:
kmer_to_off_target_count

In [ ]:
ok_candidates = {candidate for candidate, count in kmer_to_off_target_count.items() if count == 0}

In [ ]:
len(ok_candidates)

In [ ]:
ok_candidates

In [23]:

with open("aso-candidates-%dmer.txt" % aso_size, "w") as f:
    for seq in ok_candidates:
        f.write(seq + "\n")